In [7]:
import son_main_script as soon
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from pymoo.indicators.hv import Hypervolume
import json
import os
import plotly.graph_objects as go
import numpy as np

In [8]:
# load dataset

from cProfile import label
from re import X
from matplotlib import axis

from son_pymoo import ObjectiveEnum


def sign_objective(value: float, key: str):
    if(key == ObjectiveEnum.AVG_DL_RATE.name or 
       key == ObjectiveEnum.AVG_RSSI.name or
       key == ObjectiveEnum.AVG_SINR.name or
       key == ObjectiveEnum.ENERGY_EFFICIENCY.name):
        return -value
    else:
        return value

network_name = "delete3"
param_configuration_list =["algorithm_config_2STATIC"]
results_objectives_list = []
results_objective_spaces_list = []
result_dict_object = {"optimization_objectives": []}
ref_points = []
histories = []
for configuration_name in param_configuration_list:
    results_directory = "./datastore/{network_name}/{configuration_name}/".format(network_name=network_name, configuration_name= configuration_name)

    ## read json file 
    with open(results_directory + "/objectives_result.json", mode="r",encoding="utf-8") as openfile:
        result_dict_object = json.load(openfile)
        ref_points = [1.1 for  x in result_dict_object["optimization_objectives"]]

        results_objectives_list.append(np.array(
        [[sign_objective(y[1][key], key) for key in result_dict_object["optimization_objectives"]]
        for y in [item for item in result_dict_object["results"]]]))
        histories.append({"name": configuration_name, "history": result_dict_object["history"]})
        
        results_objective_spaces_list.append(np.array(result_dict_object["objectiveSpace"]))

## prepare diagrams
fig2 = go.Figure()
fig2.update_layout(
    title="pareto fronts",
    xaxis_title=result_dict_object["optimization_objectives"][0],
    yaxis_title=result_dict_object["optimization_objectives"][1],
    legend_title="configuration names",
)
fig1 = go.Figure()
fig1.update_layout(
    title="objective results",
    xaxis_title=result_dict_object["optimization_objectives"][0],
    yaxis_title=result_dict_object["optimization_objectives"][1],
    legend_title="configuration names",
)
fig3 = go.Figure()
fig3.update_layout(
    title="Convergence",
    xaxis_title="Function Evaluations",
    yaxis_title="Hypervolume",
    legend_title="configuration names"
)
## normalize each solution and calculate hyper volume and add dataj to diagrams
## since the ideal and nadir points and pareto front are not known we approximate them
combined_result_list = []
for _, solution in enumerate(results_objectives_list):
    for _, solution_candidate in enumerate(solution):
        combined_result_list.append(solution_candidate)
combined_result_list = np.array(combined_result_list)
approx_ideal = combined_result_list.min(axis=0)
approx_nadir  = combined_result_list.max(axis=0)
hv = Hypervolume(ref_point=np.array(ref_points),
                     norm_ref_point=False,
                     zero_to_one=True,
                     ideal=approx_ideal,
                     nadir=approx_nadir)

for index, solution in enumerate(results_objectives_list):
    normalized_solution = np.array((solution - approx_ideal) / (approx_nadir - approx_ideal))

    fig2.add_trace(go.Scatter(x=normalized_solution[:,0], y=normalized_solution[:,1],
                    mode='markers',
                    name=param_configuration_list[index]))
    
    fig1.add_trace(go.Scatter(x=solution[:,0], y=solution[:,1],
                mode='markers',
                name=param_configuration_list[index] ))

## add data to convergence diagram
## since the ideal and nadir points and pareto front are not known we approximate them
combined_result_list = []
for _, solution in enumerate(results_objective_spaces_list):
    for _, solution_candidate in enumerate(solution):
        combined_result_list.append(solution_candidate)
combined_result_list = np.array(combined_result_list)
approx_ideal = combined_result_list.min(axis=0)
approx_nadir  = combined_result_list.max(axis=0)
hv = Hypervolume(ref_point=np.array(ref_points),
                     norm_ref_point=False,
                     zero_to_one=True,
                     ideal=approx_ideal,
                     nadir=approx_nadir)


for _, history  in enumerate(histories):
    hv_list = [hv(np.array(solution)) for solution in history["history"]["objective_space_opt"]]
    
    fig3.add_trace(
        go.Scatter(
            x=history["history"]["n_evals"],
            y=hv_list,
            mode='lines+markers',
            name=history["name"]
        )
    )

fig2.show()
fig1.show()
fig3.show()



In [9]:

network_name = "delete3"
configuration_name ="algorithm_config_1LIVE"
results_directory = "./datastore/{network_name}/{configuration_name}/".format(network_name=network_name, configuration_name= configuration_name)
objective_np_array = np.array([])
with open(results_directory + "/objectives_result.json", mode="r",encoding="utf-8") as openfile:
    result_dict_object = json.load(openfile)
    objective_np_array = np.array(result_dict_object)
    objective_np_array[:, 1:] *= -1



approx_ideal = objective_np_array[:, 1:].min(axis=0)
approx_nadir  = objective_np_array[:, 1:].max(axis=0)

normalized_solutions_list = np.array((objective_np_array[:, 1:] - approx_ideal) / (approx_nadir - approx_ideal))

hv = Hypervolume(ref_point=np.array([1.1, 1.1]),
                     norm_ref_point=False,
                     zero_to_one=False,
                     ideal=approx_ideal,
                     nadir=approx_nadir)

hv_list = [hv(solution) for solution in normalized_solutions_list]


fig4 = go.Figure()

fig4.update_layout(
    title="hypervolume",
    xaxis_title="time in s",
    yaxis_title="Hypervolume",
    legend_title="dataset")
fig4.add_trace(
        go.Scatter(
            x=objective_np_array[:, 0],
            y=hv_list,
            mode='lines+markers',
        ))
fig5 = go.Figure()

fig5.update_layout(
    title="normalized objectives",
    xaxis_title="energy_efficiency",
    yaxis_title="avg_dl_datarate",
    legend_title="dataset")

fig5.add_trace(
        go.Scatter(
            x=normalized_solutions_list[:, 0],
            y=normalized_solutions_list[:, 1],
            text=[str(elem) for elem in objective_np_array[:,0]],
            mode='markers',
        ))
fig6 = go.Figure()

fig6.update_layout(
    title="normalized objectives",
    xaxis_title="time in s",
    yaxis_title="inverted normalized objectives",
    legend_title="dataset")

fig6.add_trace(
        go.Scatter(
            x=objective_np_array[:, 0],
            y=normalized_solutions_list[:, 0],
            name="energy_efficiency",
            mode='lines+markers',
        ))
fig6.add_trace(
        go.Scatter(
            x=objective_np_array[:, 0],
            y=normalized_solutions_list[:, 1],
            name="avrg_dl_datarate",
            mode='lines+markers',
        ))

fig4.show()
fig5.show()
fig6.show()